# **Google Data Analytics Capstone Project**


**Analyzed by**: Tibebu Sime

**Date**: 03/05/2022



### **Objective**

This capstone project is a part of the Google Data Analytics Professional Certification. The objective of this data analysis is to design marketing strategies aimed at converting casual riders into annual members. Casual riders are customers who purchase single-ride or full-day passes while members are customers who purchase annual memberships.
Business Task
The new marketing strategy requires us to understand the following three components: behavioral differences between annual members and casual riders, reasons why a casual rider would buy Cyclistic annual memberships, and how digital media can influence casual riders to become members. The marketing analytics team is tasked with using past historical trip data to find the behavioral differences between annual members and casual riders and report their findings to the key stakeholders.

### **Stakeholders**

1. ***Cyclistic***: A bike-share company based out of the City of Chicago that features more than 5800 bicycles and 600 docking stations.
2. ***Lily Moreno***: The director of marketing department who is responsible for the development of campaigns and initiatives to promote the bike-share program through email, social media & other channels.
3. ***Cyclistic Marketing Analytics Team***: A team of data analysts who are responsible for collecting, analyzing and reporting data that helps guide Cyclistic marketing strategy.
4. ***Cyclistic Executive Team***: The detail-oriented executive team who will decide whether to approve the recommended marketing program. 

### **Data Source**

The data has been made available by Motivate International Inc. & the City of Chicago under this [license](https://ride.divvybikes.com/data-license-agreement). This dataset contains the previous 12 months of trip data from Chicago's Divvy Ride Share Service for the period January to December 2021. The dataset can be downloaded from [here](https://divvy-tripdata.s3.amazonaws.com/index.html). All riders' personal information has been removed for privacy issues.

### **Data Dictionary**

1. `ride_id`: unique id of each ride
2. `rideable_type`: category of bike type such as classic, electric and docked
3. `started_at`: start time of ride
4. `ended_at`: end time of ride
5. `start_station_name`: name of station the ride started at
6. `start_station_id`: unique id of station the ride started at
7. `end_station_name`: name of station the ride ended at
8. `end_station_id`: unique id of station the ride ended at
9. `start_lat`: latitudinal coordinate of start station
10. `start_lng`: longitudinal coordinate of start station
11. `end_lat`: latitudinal coordinate of end station
12. `end_lng`: longitudinal coordinate of end station
13. `member_casual`: category of rider type (casual, member)


# **A. Data Preparation and Cleaning**

In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
library(tidyverse) 
library(lubridate)
library(janitor)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
list.files(path = "../input")

In [2]:
# Read the previous 12 months of the Cyclistic trip data 
tripdata1 <- read_csv("../input/divvy-tripdata/202101-divvy-tripdata.csv")
tripdata2 <- read_csv("../input/divvy-tripdata/202102-divvy-tripdata.csv")
tripdata3 <- read_csv("../input/divvy-tripdata/202103-divvy-tripdata.csv")
tripdata4 <- read_csv("../input/divvy-tripdata/202104-divvy-tripdata.csv")
tripdata5 <- read_csv("../input/divvy-tripdata/202105-divvy-tripdata.csv")
tripdata6 <- read_csv("../input/divvy-tripdata/202106-divvy-tripdata.csv")
tripdata7 <- read_csv("../input/divvy-tripdata/202107-divvy-tripdata.csv")
tripdata8 <- read_csv("../input/divvy-tripdata/202108-divvy-tripdata.csv")
tripdata9 <- read_csv("../input/divvy-tripdata/202109-divvy-tripdata.csv")
tripdata10 <- read_csv("../input/divvy-tripdata/202110-divvy-tripdata.csv")
tripdata11 <- read_csv("../input/divvy-tripdata/202111-divvy-tripdata.csv")
tripdata12 <- read_csv("../input/divvy-tripdata/202112-divvy-tripdata.csv")

In [3]:
# Combine the datasets into one using the bind_rows() from the dplyr package
tripdata <- bind_rows(tripdata1,
                      tripdata2,
                      tripdata3,
                      tripdata4,
                      tripdata5,
                      tripdata6,
                      tripdata7,
                      tripdata8,
                      tripdata9,
                      tripdata10,
                      tripdata11,
                      tripdata12)

In [4]:
# Quick overview of the dataset
head(tripdata, 2)

In [7]:
# Display the shape of the dataset
dim(tripdata)

In [8]:
# Set the column ride_id as row names because the values of this column are unique
# https://stackoverflow.com/questions/20643166/set-a-data-frame-column-as-the-index-of-r-data-frame-object
tripdata <- tripdata %>% column_to_rownames(var = "ride_id")
head(tripdata, 2)

In [9]:
# Remove duplicate values
tripdata <- distinct(tripdata)

In [10]:
# Remove missing values - all the three codes do the same thing 
tripdata <- drop_na(tripdata) # from tydr packagere
tripdata <- remove_missing(tripdata) # from ggplot2 package
tripdata <- remove_empty(tripdata, which = c("rows", "cols"), quiet = TRUE)  # from janitor package

In [11]:
# Check the list of column names
colnames(tripdata)

# **B. Data Transformation & Feature Engineering**

In [12]:
# https://shanghai.hosting.nyu.edu/data/r/dates-and-times.html
# https://rdrr.io/r/base/strptime.html
# https://stackoverflow.com/questions/9839343/extracting-time-from-posixct
# https://www.marsja.se/how-to-extract-time-from-datetime-in-r-with-examples/
# https://stackoverflow.com/questions/9216138/find-the-day-of-a-week

tripdata <- tripdata %>% mutate(date = strftime(started_at, format = "%Y-%m-%d"),
                                month = strftime(started_at, format = "%B"),
                                day = strftime(started_at, format = "%d"),
                                year = strftime(started_at, format = "%Y"),
                                weekday = strftime(started_at, format = "%A"),
                                time = strftime(started_at, format = "%H:%M"))

tripdata$date <- as.Date(tripdata$date)
tripdata$time <- as.POSIXct(tripdata$time, format="%H:%M")

In [13]:
# Now check the structure of the columns
str(tripdata)

In [21]:
# Select only columns necessary for our data analysis, and calculate the length of ride started
# different stations. Here the ride_length is measured in seconds!
tripdata <- tripdata %>% 
  select(rideable_type, started_at, ended_at, member_casual, date, month, day, year, weekday, time) %>% 
  mutate(ride_length = ended_at-started_at)

In [24]:
# Quick overview of the dataset again
head(tripdata, 2)

In [25]:
# Convert ride_length from Factor to numeric so that we can run calculations on the data
# The trip duration should be measured in minutes for easy understanding
tripdata$ride_length <- as.double(tripdata$ride_length)/60

In [26]:
head(tripdata, 2)

In [27]:
# Inspect the structure of the selected and manipulated columns again
str(tripdata)

**Remove bad data or outliers**

The dataframe includes a hundred entries when bikes were taken out docks and checked for quality by Cyclistic. Negative rides and rides longer than 1 day or 1440 minutes or 86400 seconds should be filtered out.

In [28]:
tripdata <- filter(tripdata, ride_length > 0 & ride_length <= 1440)

In [29]:
# Rename the column names for better readability
tripdata <- tripdata %>% rename(bike_type = rideable_type, 
                                customer_type = member_casual,
                                start_time = started_at,
                                end_time = ended_at)

In [30]:
# Since the days of week are out of order, we need to convert them to an ordered factor 
# so that our data analysis will be consistent.
# https://stackoverflow.com/questions/10309564/reorder-factor-levels-by-day-of-the-week-in-r

tripdata$weekday <- ordered(tripdata$weekday, levels=c("Monday", 
                                                       "Tuesday", 
                                                       "Wednesday",
                                                       "Thursday", 
                                                       "Friday", 
                                                       "Saturday", 
                                                       "Sunday"))


tripdata$month <- ordered(tripdata$month, levels=c("January", 
                                                   "February", 
                                                   "March", 
                                                   "April",
                                                   "May", 
                                                   "June", 
                                                   "July", 
                                                   "August",
                                                   "September",
                                                   "October",
                                                   "November", 
                                                   "December"))

In [31]:
# Quick overview
head(tripdata, 2)

# **C. Data Analysis**

In [32]:
# Check the number of each customer type 
table(tripdata$customer_type)

In [33]:
# Let us check the proportion of annual members and casual riders in percentage
table(tripdata$customer_type)/nrow(tripdata)*100

In [34]:
# Show the statistical summary of the trip duration for all riders
summary(tripdata$ride_length)

In [35]:
# Compare members and casual riders using min, max and average of trip duration
tripdata %>% 
  group_by(customer_type) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            total_ride = sum(ride_length, na.rm = TRUE))

From the above statistical summary of the trip duration for all customers and by the customer type, we can see that the average trip duration for the member riders is lower than the average trip duration for all riders while the avg trip duration of the casual riders is higher than that for all riders. This indicates that the casual riders usually take a longer trip compared the member riders. 

In [36]:
# Determine the average trip duration in seconds by each day for members and casual riders
# https://rstats-tips.net/2020/07/31/get-rid-of-info-of-dplyr-when-grouping-summarise-regrouping-output-by-species-override-with-groups-argument/
tripdata %>% group_by(customer_type, weekday) %>% 
                  summarize(min_ride = min(ride_length, na.rm = TRUE),
                            max_ride = max(ride_length, na.rm = TRUE),
                            mid_ride = median(ride_length, na.rm = TRUE),
                            avg_ride = mean(ride_length, na.rm = TRUE),
                            num_of_rides = n(),
                            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
                                  arrange(weekday)

In [37]:
# Calculate the statistical summary of the trip duration by each month for members & casual riders
tripdata %>% group_by(customer_type, month) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
  arrange(month)

In [38]:
# Calculate the statistical summary of the bike demand over 24 hours of the day
tripdata %>% group_by(customer_type, time) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
  arrange(time)

# **D. Data Visualizations**

**1. Creating visualization for the number of rides each day of week by rider type**

In [56]:
# Let's visualize the number of rides each day of week by rider type
# https://www.kaggle.com/getting-started/105201
# https://www.kaggle.com/regressionsquirrel/resize-ggplot
# https://stackoverflow.com/questions/34522732/changing-fonts-in-ggplot2
# https://stackoverflow.com/questions/28243514/ggplot2-change-title-size

options(repr.plot.width = 12, repr.plot.height = 8) # used to resize ggplot figure
tripdata %>% group_by(customer_type, weekday) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
  arrange(weekday) %>% 
  ggplot(mapping = aes(x=weekday, y=num_of_rides, fill=customer_type))+
  geom_bar(stat = "identity", position = "dodge")+
  scale_fill_manual(values = c("orange", "darkblue"))+
  labs(x = "Day of Week",
       y = "Number of Trips", 
       title = "Total Trips per Day By Customer Type",
       fill = "Customer Type")+
  theme(plot.title =element_text(family = "serif", color = "darkred", size = 18, face="bold", hjust=0.5),
        axis.title.x = element_text(face="bold", size = 12),
        axis.text.x  = element_text(face = "bold", size = 10),
        axis.title.y = element_text(face = "bold", size = 12),
        axis.text.y = element_text(face = "bold", size = 10),
        legend.title = element_text(face = "bold", size = 12),
        legend.text = element_text(face = "bold", size = 10))+
  scale_y_continuous(labels = function(y)format(y, scientific=FALSE))

From the bar graph above, we can observe that casual riders are highly active on weekends because it is obvious that casual riders mostly use bike-sharing service for leisure and tourism purposes. It is interesting to note that the distribution of the number of trips among the annual members is more or less consistent over the entire but slightly goes down on weekends. This indicates members mainly use the bike-sharing services to commute work.

**2. Creating visualization for average trip duration each day by rider type**

In [55]:
options(repr.plot.width = 12, repr.plot.height = 8)
tripdata %>% group_by(customer_type, weekday) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
  arrange(weekday) %>% 
  ggplot(mapping = aes(x=weekday, y=avg_ride, fill=customer_type))+
  geom_bar(stat = "identity", position = "dodge")+
  scale_fill_manual(values = c("orange", "darkblue"))+
  labs(x = "Day of Week",
       y = "Average Trip Duration", 
       title = "Average Trip Duration per Day By Customer Type",
       fill = "Customer Type")+
   theme(plot.title =element_text(family = "serif", color = "darkred", size = 18, face="bold", hjust=0.5),
         axis.title.x = element_text(face="bold", size = 12),
         axis.text.x  = element_text(face = "bold", size = 10),
         axis.title.y = element_text(face = "bold", size = 12),
         axis.text.y = element_text(face = "bold", size = 10),
         legend.title = element_text(face = "bold", size = 12),
         legend.text = element_text(face = "bold", size = 10))+
  scale_y_continuous(labels = function(y)format(y, scientific=FALSE))

The bar graph above shows that the average trip duration of casual riders over the entire week is considerably longer than or more than twice that of the annual members, and takes a high peak on weekends. The distribution of the average trip duration for the members is consistent irrespective of the day of week.

**3. Creating visualization for the number of rides each month by rider type**

In [60]:
options(repr.plot.width = 12, repr.plot.height = 8)
tripdata %>% group_by(customer_type, month) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
  arrange(month) %>% 
  ggplot(mapping = aes(x=month, y=num_of_rides, fill=customer_type))+
  geom_bar(stat = "identity", position = "dodge")+
  scale_fill_manual(values = c("darkred", "darkgreen"))+
  labs(x = "Month",
       y = "Number of Trips", 
       title = "Total Trips per Month By Customer Type",
       fill = "Customer Type")+
   theme(plot.title =element_text(family = "serif", color = "darkblue", size = 18, face="bold", hjust=0.5),
         axis.title.x = element_text(face="bold", size = 12),
         axis.text.x  = element_text(face = "bold", size = 10, angle = 45),
         axis.title.y = element_text(face = "bold", size = 12),
         axis.text.y = element_text(face = "bold", size = 10),
         legend.title = element_text(face = "bold", size = 12),
         legend.text = element_text(face = "bold", size = 10))+
  scale_y_continuous(labels = function(y)format(y, scientific=FALSE))

From the graph above, we can see that summer is the busiest time of the year among both annual and casual riders but the number of trips taken by the casual riders is higher than the annual members in June, July & August. The lean period of the year (November through March) shows the same trend among both type of riders but the casual riders’ bike usage is very low during this season compared to that of the annual members which is fairly good. This could be attributed to multiple factors such as bad weather condition that might impair the customers from using bikes. 

**4. Creating visualization for average trip duration each month by rider type**

In [68]:
options(repr.plot.width = 12, repr.plot.height = 8)
tripdata %>% group_by(customer_type, month) %>% 
  summarize(min_ride=min(ride_length, na.rm = TRUE),
            max_ride=max(ride_length, na.rm = TRUE),
            mid_ride=median(ride_length, na.rm = TRUE),
            avg_ride=mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride=sum(ride_length, na.rm = TRUE)) %>% 
  arrange(month) %>% 
  ggplot(mapping = aes(x=month, y=avg_ride, fill=customer_type))+
  geom_bar(stat = "identity", position = "dodge")+
  scale_fill_manual(values = c("darkred", "darkgreen"))+
  labs(x = "Month",
       y = "Average Trip Duration", 
       title = "Average Trip Duration per Month By Customer Type",
       fill = "Customer Type")+
  theme(plot.title =element_text(family = "serif", color = "darkblue", size = 18, face="bold", hjust=0.5),
        axis.title.x = element_text(face="bold", size = 12),
        axis.text.x  = element_text(face = "bold", size = 10, angle = 45),
        axis.title.y = element_text(face = "bold", size = 12),
        axis.text.y = element_text(face = "bold", size = 10),
        legend.title = element_text(face = "bold", size = 12),
        legend.text = element_text(face = "bold", size = 10))+
  scale_y_continuous(labels = function(y)format(y, scientific=FALSE))

The graph above shows that the average trip duration made by casual riders is still considerably longer than those of the annual members throughout the year.

**5. Creating visualization for bike demand throughout the day**

In [72]:
tripdata %>% group_by(customer_type, time) %>% 
  summarize(min_ride = min(ride_length, na.rm = TRUE),
            max_ride = max(ride_length, na.rm = TRUE),
            mid_ride = median(ride_length, na.rm = TRUE),
            avg_ride = mean(ride_length, na.rm = TRUE),
            num_of_rides = n(),
            total_ride = sum(ride_length, na.rm = TRUE)) %>% 
  arrange(time) %>% 
  ggplot(mapping = aes(x=time, y=num_of_rides, color=customer_type, group=customer_type))+
  geom_line(size=0.8)+
  scale_x_datetime(date_breaks = "1 hour", date_labels = "%H:%M", expand = c(0,0))+
  labs(x = "Time of Day",
       y = "Number of Trips", 
       title = "Bike Demand Throughout The Day",
       color = "Customer Type")+
 theme(plot.title =element_text(family = "serif", color = "darkblue", size = 18, face="bold", hjust=0.5),
       axis.title.x = element_text(face="bold", size = 12),
       axis.text.x  = element_text(face = "bold", size = 10, angle = 45),
       axis.title.y = element_text(face = "bold", size = 12),
       axis.text.y = element_text(face = "bold", size = 10),
       legend.title = element_text(face = "bold", size = 12),
       legend.text = element_text(face = "bold", size = 10))+
  scale_y_continuous(labels = function(y)format(y, scientific=FALSE))

The bike demand throughout the day shows that the usage by the members is high during the rush hours and takes a sharp drop after it peaks at 5 PM, which confirms our assumption that the majority of annual members are office-goers who use bikes to commute to work. But the bike usage by the casual riders steadily increases throughout the day and starts to steadily decrease after it peaks at 5 PM.

**6. Creating visualization for the number of each bike type used the riders**

In [74]:
tripdata %>% group_by(customer_type, bike_type) %>% 
    summarize(total_rides = n()) %>% 
    ggplot(mapping = aes(x=bike_type, y=total_rides, fill=customer_type))+
    geom_bar(stat="identity", position = "dodge")+
    scale_fill_manual(values = c("red", "blue"))+
    labs(x = "Bike Type",
         y = "Number of Rides", 
         title = "Bike Type Usage By The Customers",
         fill = "Customer Type")+
    theme(plot.title =element_text(family = "serif", color = "black", size = 18, face="bold", hjust=0.5),
          axis.title.x = element_text(face="bold", size = 12),
          axis.text.x  = element_text(face = "bold", size = 10),
          axis.title.y = element_text(face = "bold", size = 12),
          axis.text.y = element_text(face = "bold", size = 10),
          legend.title = element_text(face = "bold", size = 12),
          legend.text = element_text(face = "bold", size = 10))+
    scale_y_continuous(labels = function(y)format(y, scientific=FALSE))

From the graph, we can see that members use classic bikes much more than the casual riders do.  The distribution of the use of electric is almost identical for both customers but docked bikes are more preferred by the casual riders. 

# **E. Conclusion**

### **Key Takeaways**
1. Casual riders take bike rides more than double that of annual members on average
2. Casual riders use bike-sharing services more over the weekends for leisure while members use them consistently throughout the entire week to commute to work.
3. Annual members predominately use classic bikes and rarely use docked bikes while casual riders are open to all kinds of bikes but prefer docked bikes.


### **Recommendations**

1. Lower the price of single-ride and full-day tickets casual rides so that they will be enticed to use bikes during the weekdays
2. Offer attractive promotions like 20% to 30 % discount on the first year subscription to make casual riders become annual members 
3. Give a two-week free trial for casual riders who want to first try the bike-sharing service before committing to the annual plan.


### **Additional Data for Further Analysis**

1. Plans & pricing: this data can be used to optimize the cost benefit analysis for existing customers and potential new customers.  Consider creating a seasonal special price for summer and winter seasons
2. Age and gender profile: this data can be used to study the age and gender category of the riders so that the marketing team will efficiently use their budget to target potential customers
3. Neighborhood details: this can be used to investigate the riders’ bike usage based on the residential areas so that the marketing team create promotions or make advertisements accordingly. 
